In [93]:
#| label: bill-ratio

import string
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from typing import Callable
from fastcore.all import *


def initialize_random_parameters(input_size: int, output_size: int) -> nn.Module:
    """
    Initialize a model with random weights and bias.
    """
    model = nn.Linear(input_size, output_size)
    nn.init.uniform_(model.weight, -0.1, 0.1)
    nn.init.uniform_(model.bias, -0.1, 0.1)
    return model

class Learner:
    """
    A simple learning framework for training PyTorch models.
    """
    def __init__(self, 
                 model: nn.Module, 
                 loss_func: Callable, 
                 optimizer: 'Optimizer', 
                 data: DataLoader):
        self.model = model
        self.loss_func = loss_func
        self.optimizer = optimizer
        self.data = data

    def fit(self, epochs: int) -> None:
        """
        Fit the model to the data for a number of epochs.
        """
        for epoch in range(epochs):
            for xb, yb in self.data:
                self.train_step(xb, yb)
                self.optimizer.step()

    def train_step(self, xb: torch.Tensor, yb: torch.Tensor) -> None:
        """
        Perform a single training step: forward pass and loss calculation.
        """
        preds = self.model(xb)
        loss = self.loss_func(preds, yb)
        loss.backward()
        print("loss", loss.mean())

class Model:
    def __init__(self, size) -> None:
        self.weights = (torch.randn(size)*1.0).requires_grad_()
        self.bias= (torch.randn(size)*1.0).requires_grad_()
        self.params = self.weights, self.bias
    
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        # print(x.shape, self.weights.shape, self.bias.shape)
        # print(x.type(), self.weights.type(), self.bias.type())
        return x @ self.weights + self.bias

class Optimizer:
    """
    A basic optimizer that updates model parameters using gradient descent.
    """
    def __init__(self, model: 'Model', lr: float):
        self.model = model
        self.lr = lr
  
    def step(self) -> None:
        """
        Update the parameters based on the gradients.
        """
        for p in self.model.params:
            p.data -= p.grad * self.lr
            p.grad.zero_()




In [94]:
# Example usage
from torch import tensor


input_size = (26)  # Define the input size

#model = initialize_random_parameters(input_size, output_size)
model = Model(input_size)
optimizer = Optimizer(model, lr=0.1)
ds = [(tensor(x*1.0),tensor(ord(y)*1.0)) for x,y in enumerate(string.ascii_lowercase)]
# ds

In [95]:
data = DataLoader(ds, batch_size=26, shuffle=True)
def loss_func(preds, targets):
    return ((preds-targets)**2).mean()

learner = Learner(model, loss_func, optimizer, data)


In [96]:
print(model.params)
learner.fit(10)
model.params

(tensor([-1.0595,  0.5252,  1.0667,  0.1389,  0.8173,  0.3031,  0.7115,  0.9100,
         0.6401, -0.8909, -1.7828,  1.3262,  0.4102, -0.2634, -1.0469, -0.0982,
        -0.2595,  0.2958, -0.3669, -0.2187,  0.7747,  1.0876, -1.4512,  0.2850,
         1.3892,  0.0906], requires_grad=True), tensor([ 0.2880, -0.2736,  1.8074,  0.3579, -0.2038, -0.0209, -0.4879, -0.9611,
        -0.5916, -1.6469, -1.2140, -0.3337,  1.6956,  1.1160,  0.8451, -0.1683,
         1.4993, -1.0675,  0.1393,  1.0348,  0.5034,  0.8949,  0.5711, -0.6498,
         0.7184,  0.3145], requires_grad=True))
loss tensor(5788.5679, grad_fn=<MeanBackward0>)
loss tensor(3.4479e+09, grad_fn=<MeanBackward0>)
loss tensor(2.9855e+15, grad_fn=<MeanBackward0>)
loss tensor(1.7222e+21, grad_fn=<MeanBackward0>)
loss tensor(6.7011e+26, grad_fn=<MeanBackward0>)
loss tensor(3.7286e+32, grad_fn=<MeanBackward0>)
loss tensor(inf, grad_fn=<MeanBackward0>)
loss tensor(inf, grad_fn=<MeanBackward0>)
loss tensor(inf, grad_fn=<MeanBackward0>)
loss

(tensor([-1.9110e+28, -3.2472e+28, -3.0590e+28, -2.8623e+28, -4.0101e+28,
         -4.3960e+28, -4.7744e+28, -2.1015e+28, -3.4403e+28, -2.6720e+28,
         -3.6305e+28, -3.8054e+27, -1.5261e+28, -5.6781e+27, -1.1430e+28,
         -4.5835e+28, -1.3360e+28, -1.9096e+27, -9.5157e+27, -4.2055e+28,
         -2.4815e+28, -3.8206e+28, -7.6222e+27, -1.7179e+28, -2.2938e+28,
          3.6884e+25], requires_grad=True),
 tensor([-7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25,
         -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25,
         -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25,
         -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25,
         -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25, -7.3446e+25,
         -7.3446e+25], requires_grad=True))